# Data Cleaning and extraction (LAMIA)


In [1]:
import pandas as pd
import json
from os import listdir
from os.path import isfile, join

In [2]:
#df_nested_list = pd.json_normalize(data, record_path =['gameData'])
def get_generalData(data):
    id_game = data['gameData']['game']['pk']
    season = data['gameData']['game']['season']
    try :
        dateTime =  data['gameData']['datetime']['dateTime']
    except :
        print(f"dateTime not found for match {id_game}")
        dateTime = None
    try :
        endDateTime = data['gameData']['datetime']['endDateTime']
    except :
        #print(f"endDateTime not found for match {id_game}")
        endDateTime = None
    
    abstractGameState = data['gameData']['status']['abstractGameState']
    team_away_name = data['gameData']['teams']['away']['name']
    team_home_name = data['gameData']['teams']['home']['name']
    return [id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name]

In [3]:
def get_file_event_rows_data(data,type_season):
    match_data = get_generalData(data)
    match_events_list = []
    for item in data['liveData']['plays']['allPlays']:
        if item['result']['event'] not in ["Goal", "Shot"]:
            continue
        else :
            # event Idx
            event_Idx = item['about']['eventIdx']

            # periodTime
            periodTime = item['about']['periodTime']
            
            # team information (which team shot)
            id_team_that_shot = item['team']['id'] 
            name_team_that_shot = item['team']['name'] 

            # indicator if its a shot or a goal
            result_event = item['result']['event']
            
            # the on ice coordinates
            try :
                x_coord = item['coordinates']['x'] 
                y_coord = item['coordinates']['y']
            except :
                print(f"coordinates not found for match {match_data[-7]} and event {event_Idx}")
                x_coord = None
                y_coord = None
            
            goalie_name = None
            shooter_name = None
            # the shooter and goalie name
            for item_bis in item['players']:
                if item_bis['playerType']=="Goalie":
                    goalie_name = item_bis['player']["fullName"]
                elif item_bis['playerType'] in ["Shooter", "Scorer"]:
                    shooter_name = item_bis['player']["fullName"]
                else:
                    continue
            '''
            if goalie_name == None :
                print(f"goalie_name not found for match {match_data[-7]} and event {event_Idx}")
            if shooter_name == None :
                print(f"shooter_name  not found for match {match_data[-7]} and event {event_Idx}")
            '''
            # shot type
            try :
                shot_type = item['result']['secondaryType']
            except :
                # sometimes the secondary Type is not defined
                print(f"shot_type not found for match {match_data[-7]} and event {event_Idx}")
                shot_type = None

            try : 
                # empty net
                empty_net = item['result']['emptyNet']
            except :
                empty_net = None

            
            # strength
            try :
                strength = item['result']['name']
            except :
                strength = None
            
            all_data = [event_Idx, periodTime, id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,goalie_name,shooter_name,shot_type,empty_net,strength]
            match_events_list.append(all_data+[type_season]+match_data)
    return match_events_list


 

In [4]:
dir_year = [ join("../datasets/raw/", d_y) for d_y in listdir("../datasets/raw/")]
dir_pl_reg = [(join(path, d_pl_reg),d_pl_reg)  for path in dir_year for d_pl_reg in listdir(path)  ]
fichiers = [(join(dir_path, f),d_pl_reg) \
    for (dir_path, d_pl_reg) in dir_pl_reg
    for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [5]:
all_list_data = []
for (file_name_path, type_season) in fichiers :
    with open(file_name_path,'r') as f:
        #print(f)
        data = json.loads(f.read())
        all_list_data = all_list_data + get_file_event_rows_data(data,type_season)

coordinates not found for match 2017020533 and event 303
coordinates not found for match 2017020533 and event 304
coordinates not found for match 2017020533 and event 306
coordinates not found for match 2017020533 and event 305
coordinates not found for match 2017020533 and event 307
coordinates not found for match 2017020387 and event 391
shot_type not found for match 2017021124 and event 255
shot_type not found for match 2017020500 and event 320
shot_type not found for match 2017020966 and event 342
coordinates not found for match 2017020273 and event 218
coordinates not found for match 2017020642 and event 383
shot_type not found for match 2019020776 and event 175
shot_type not found for match 2019020873 and event 322
shot_type not found for match 2019020809 and event 278
shot_type not found for match 2019020575 and event 335
coordinates not found for match 2019020349 and event 351
shot_type not found for match 2019020349 and event 351
shot_type not found for match 2019020451 and ev

In [ ]:
columns_name = [
"event_Idx", "periodTime","id_team_that_shot","name_team_that_shot","result_event","x_coord","y_coord","goalie_name",\
        "shooter_name","shot_type","empty_net","strength",\
        "type_season","id_game","season","dateTime","endDateTime","abstractGameState","team_away_name","team_home_name"]
df = pd.DataFrame(all_list_data, columns=columns_name)

In [ ]:
df.head(5)

,event_Idx,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,goalie_name,shooter_name,shot_type,empty_net,strength,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
0,4,00:13,23,Vancouver Canucks,Shot,-78.0,4.0,Martin Jones,Sam Gagner,Wrist Shot,None,None,regular_season,2017020495,20172018,2017-12-16T03:00:00Z,2017-12-16T05:38:03Z,Final,San Jose Sharks,Vancouver Canucks
1,5,00:14,23,Vancouver Canucks,Shot,-81.0,0.0,Martin Jones,Sam Gagner,Wrist Shot,None,None,regular_season,2017020495,20172018,2017-12-16T03:00:00Z,2017-12-16T05:38:03Z,Final,San Jose Sharks,Vancouver Canucks
2,9,00:44,23,Vancouver Canucks,Goal,-79.0,-1.0,Martin Jones,Markus Granlund,Deflected,False,None,regular_season,2017020495,20172018,2017-12-16T03:00:00Z,2017-12-16T05:38:03Z,Final,San Jose Sharks,Vancouver Canucks
3,11,01:20,23,Vancouver Canucks,Shot,-67.0,20.0,Martin Jones,Brock Boeser,Wrist Shot,None,None,regular_season,2017020495,20172018,2017-12-16T03:00:00Z,2017-12-16T05:38:03Z,Final,San Jose Sharks,Vancouver Canucks
4,15,02:08,23,Vancouver Canucks,Shot,-38.0,-1.0,Martin Jones,Alex Biega,Slap Shot,None,None,regular_season,2017020495,20172018,2017-12-16T03:00:00Z,2017-12-16T05:38:03Z,Final,San Jose Sharks,Vancouver Canucks


In [ ]:
df[df.goalie_name.isna()]

,event_Idx,periodTime,id_team_that_shot,name_team_that_shot,result_event,x_coord,y_coord,goalie_name,shooter_name,shot_type,empty_net,strength,type_season,id_game,season,dateTime,endDateTime,abstractGameState,team_away_name,team_home_name
668,322,18:47,54,Vegas Golden Knights,Goal,-12.0,-12.0,None,David Perron,Wrist Shot,True,None,regular_season,2017020568,20172018,2017-12-28T03:00:00Z,2017-12-28T05:36:17Z,Final,Vegas Golden Knights,Anaheim Ducks
727,303,16:52,10,Toronto Maple Leafs,Goal,-19.0,12.0,None,William Nylander,Wrist Shot,True,None,regular_season,2017020092,20172018,2017-10-18T23:30:00Z,2017-10-19T02:15:51Z,Final,Detroit Red Wings,Toronto Maple Leafs
858,308,19:11,8,Montréal Canadiens,Goal,-78.0,-1.0,None,Alex Galchenyuk,Wrist Shot,True,None,regular_season,2017020991,20172018,2018-03-03T00:00:00Z,2018-03-03T02:35:16Z,Final,Montréal Canadiens,New York Islanders
1422,302,19:38,10,Toronto Maple Leafs,Goal,51.0,27.0,None,Nazem Kadri,Wrist Shot,True,None,regular_season,2017020084,20172018,2017-10-17T23:00:00Z,2017-10-18T01:32:56Z,Final,Toronto Maple Leafs,Washington Capitals
1479,268,17:40,18,Nashville Predators,Goal,71.0,33.0,None,Viktor Arvidsson,Wrist Shot,True,None,regular_season,2017020591,20172018,2017-12-31T01:00:00Z,2017-12-31T03:29:57Z,Final,Minnesota Wild,Nashville Predators
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389008,312,19:16,8,Montréal Canadiens,Goal,-3.0,40.0,None,Torrey Mitchell,Wrist Shot,True,None,playoffs,2016020102,20162017,2016-10-27T23:30:00Z,2016-10-28T02:11:16Z,Final,Tampa Bay Lightning,Montréal Canadiens
389132,376,18:40,28,San Jose Sharks,Goal,74.0,-23.0,None,Joe Thornton,None,True,None,playoffs,2016020122,20162017,2016-10-30T02:30:00Z,2016-10-30T05:06:35Z,Final,Nashville Predators,San Jose Sharks
389453,304,19:32,6,Boston Bruins,Goal,29.0,2.0,None,Dominic Moore,Slap Shot,True,None,playoffs,2016020227,20162017,2016-11-14T00:00:00Z,2016-11-14T02:39:06Z,Final,Boston Bruins,Colorado Avalanche
389648,317,19:46,29,Columbus Blue Jackets,Goal,-34.0,-30.0,None,Sam Gagner,Wrist Shot,True,None,playoffs,2016020406,20162017,2016-12-10T00:30:00Z,2016-12-10T03:13:24Z,Final,Columbus Blue Jackets,Detroit Red Wings
